# Run EHR-QC

## Read individual files and merge

In [1]:
import os

import pandas as pd


dfs = []
casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'
observationsDirPath = casestudy1DirPath + '/' + 'ehr_observations'
for file in os.listdir(observationsDirPath):
    df = pd.read_csv(observationsDirPath + '/' + file)
    dfs.append(df)
mergedDf = pd.concat(dfs)
mergedDf = mergedDf.drop(columns=['encounter_id'])
mergedDf


,observation_id,patient_id,observation_type_id,observation_type,value,effectiveDateTime
0,154175191,2122175,71960002,Eosinophils,0.12,2018-12-07
1,154197574,2122175,5540006,Calcium,2.18,2018-12-07
2,154168027,2122175,56972008,Lymphocytes,0.64,2018-12-07
3,9802390,2122175,246508008,Temperature Tympanic,36.10,2018-12-07
4,9787647,2122175,364075005,Heart Rate Monitored,71.00,2018-12-07
...,...,...,...,...,...,...
1163,17203364,2161432,246508008,Temperature Temporal,36.00,2019-08-30
1164,17203409,2161432,271650006,"Diastolic Blood Pressure, Cuff",60.00,2019-08-30
1165,17203655,2161432,271649006,"Systolic Blood Pressure, Cuff",119.00,2019-08-30
1166,17203361,2161432,431314004,SpO2,99.00,2019-08-30


## Save merged file

In [2]:
casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'

mergedDf.to_csv(casestudy1DirPath + '/merged_observations.csv', index=False)

## Read merged file

In [3]:
import pandas as pd


casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'

mergedDf = pd.read_csv(casestudy1DirPath + '/merged_observations.csv')
mergedDf

,observation_id,patient_id,observation_type_id,observation_type,value,effectiveDateTime
0,154175191,2122175,71960002,Eosinophils,0.12,2018-12-07
1,154197574,2122175,5540006,Calcium,2.18,2018-12-07
2,154168027,2122175,56972008,Lymphocytes,0.64,2018-12-07
3,9802390,2122175,246508008,Temperature Tympanic,36.10,2018-12-07
4,9787647,2122175,364075005,Heart Rate Monitored,71.00,2018-12-07
...,...,...,...,...,...,...
513339,17203364,2161432,246508008,Temperature Temporal,36.00,2019-08-30
513340,17203409,2161432,271650006,"Diastolic Blood Pressure, Cuff",60.00,2019-08-30
513341,17203655,2161432,271649006,"Systolic Blood Pressure, Cuff",119.00,2019-08-30
513342,17203361,2161432,431314004,SpO2,99.00,2019-08-30


## Concept coverage analysis

In [4]:
lookupDf = mergedDf[['observation_type_id', 'observation_type']]\
    .drop_duplicates()\
    .groupby(['observation_type_id'])\
    .agg({'observation_type': 'first'})\
    .reset_index()\
    .apply(lambda row: (str(row.observation_type_id), row.observation_type), result_type='expand', axis=1)\
    .rename(
        columns={0: 'observation_type_id', 1: 'observation_type'}
    )
lookupDf

,observation_type_id,observation_type
0,767002,WBC
1,5540006,Calcium
2,6797001,"Mean Arterial Pressure, Cuff"
3,14089001,RBC
4,15373003,Creatinine
...,...,...
68,1024211000000107,pH
69,1028091000000102,GGT
70,1028281000000106,Urea
71,1032081000000104,Carboxy-Hb


In [5]:
coverageDf = mergedDf\
    .groupby(['observation_type_id'])\
    .agg({'patient_id': 'nunique'})\
    .reset_index()\
    .apply(lambda row: (
        str(row.observation_type_id),
        row.patient_id,
        round(row.patient_id/len(mergedDf.patient_id.unique())*100, 2),
        ), result_type='expand', axis=1
    ).rename(
        columns={0: 'observation_type_id', 1: 'patient_count', 2: 'patient_coverage'}
    ).merge(
        lookupDf,
        how='left',
        on=['observation_type_id']
    )

In [6]:
# pd.set_option('display.max_rows', 500)
coverageDf[coverageDf.patient_coverage > 90]
# coverageDf

,observation_type_id,patient_count,patient_coverage,observation_type
0,767002,305,100.00,WBC
1,5540006,302,99.02,Calcium
2,6797001,305,100.00,"Mean Arterial Pressure, Cuff"
3,14089001,305,100.00,RBC
4,15373003,305,100.00,Creatinine
5,16378004,305,100.00,Platelets
8,30630007,305,100.00,Neutrophils
9,38151008,302,99.02,Magnesium Level
10,39972003,305,100.00,Sodium Direct
13,50213009,305,100.00,Chloride


## Transform merged data

In [7]:
pivotDf = mergedDf[mergedDf.observation_type_id.isin(coverageDf[coverageDf.patient_coverage > 90].observation_type_id.apply(lambda x: int(x)))]\
    [['observation_id', 'patient_id', 'observation_type', 'value', 'effectiveDateTime']]\
    .pivot_table(index=['observation_id', 'patient_id', 'effectiveDateTime'], columns=['observation_type'], values=['value'], aggfunc='mean')
pivotDf

value                        \
observation_type                              ALP ALT Albumin Basophils   
observation_id patient_id effectiveDateTime                               
13493          2193278    2022-09-09          NaN NaN     NaN       NaN   
22821          1710435    2021-03-02          NaN NaN     NaN       NaN   
31396          2133327    2018-11-01          NaN NaN     NaN       NaN   
53413          2468015    2021-03-02          NaN NaN     NaN       NaN   
53417          2468015    2021-03-02          NaN NaN     NaN       NaN   
...                                           ...  ..     ...       ...   
218423095      2193278    2023-08-14          NaN NaN     NaN       NaN   
218423352      2193278    2023-08-14          NaN NaN     NaN       NaN   
218423567      2193278    2023-08-14          NaN NaN     NaN       NaN   
218423568      2193278    2023-08-14          NaN NaN     NaN       NaN   
218423801      2193278    2023-08-14          NaN NaN     NaN      0.02   

                                                                           \
observation_type                            Bicarbonate Bilirubin Calcium   
observation_id patient_id effectiveDateTime                                 
13493          2193278    2022-09-09                NaN       NaN     NaN   
22821          1710435    2021-03-02                NaN       NaN     NaN   
31396          2133327    2018-11-01                NaN       NaN     NaN   
53413          2468015    2021-03-02                NaN       NaN     NaN   
53417          2468015    2021-03-02                NaN       NaN     NaN   
...                                                 ...       ...     ...   
218423095      2193278    2023-08-14                NaN       NaN     NaN   
218423352      2193278    2023-08-14                NaN       NaN     NaN   
218423567      2193278    2023-08-14                NaN       NaN     NaN   
218423568      2193278    2023-08-14                NaN       NaN     NaN   
218423801      2193278    2023-08-14                NaN       NaN     NaN   

                                                                      \
observation_type                            Chloride Chloride Direct   
observation_id patient_id effectiveDateTime                            
13493          2193278    2022-09-09             NaN             NaN   
22821          1710435    2021-03-02             NaN             NaN   
31396          2133327    2018-11-01             NaN             NaN   
53413          2468015    2021-03-02             NaN             NaN   
53417          2468015    2021-03-02             NaN             NaN   
...                                              ...             ...   
218423095      2193278    2023-08-14             NaN             NaN   
218423352      2193278    2023-08-14             NaN             NaN   
218423567      2193278    2023-08-14             NaN             NaN   
218423568      2193278    2023-08-14             NaN             NaN   
218423801      2193278    2023-08-14             NaN             NaN   

                                                        ...  \
observation_type                            Creatinine  ...   
observation_id patient_id effectiveDateTime             ...   
13493          2193278    2022-09-09               NaN  ...   
22821          1710435    2021-03-02               NaN  ...   
31396          2133327    2018-11-01               NaN  ...   
53413          2468015    2021-03-02               NaN  ...   
53417          2468015    2021-03-02               NaN  ...   
...                                                ...  ...   
218423095      2193278    2023-08-14               NaN  ...   
218423352      2193278    2023-08-14               NaN  ...   
218423567      2193278    2023-08-14               NaN  ...   
218423568      2193278    2023-08-14               NaN  ...   
218423801      2193278    2023-08-14               NaN  ...   

                      

In [8]:
pivotDf.columns = [f'{x}_{y}' for x,y in pivotDf.columns]
pivotDf = pivotDf.reset_index()
pivotDf = pivotDf.drop(columns=['observation_id'])
pivotDf

,patient_id,effectiveDateTime,value_ALP,value_ALT,value_Albumin,value_Basophils,value_Bicarbonate,value_Bilirubin,value_Calcium,value_Chloride,...,value_Systolic Blood Pressure Supine,"value_Systolic Blood Pressure, Cuff",value_Temperature Axillary,value_Temperature Nasoesophageal,value_Temperature Temporal,value_Temperature Tympanic,value_Total Hb,value_Total Protein,value_Urea,value_WBC
0,2193278,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1710435,2021-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2133327,2018-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2468015,2021-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2468015,2021-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440869,2193278,2023-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440870,2193278,2023-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440871,2193278,2023-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440872,2193278,2023-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
datamatrixDf = pivotDf.groupby(['patient_id', 'effectiveDateTime']).mean().reset_index()
datamatrixDf

,patient_id,effectiveDateTime,value_ALP,value_ALT,value_Albumin,value_Basophils,value_Bicarbonate,value_Bilirubin,value_Calcium,value_Chloride,...,value_Systolic Blood Pressure Supine,"value_Systolic Blood Pressure, Cuff",value_Temperature Axillary,value_Temperature Nasoesophageal,value_Temperature Temporal,value_Temperature Tympanic,value_Total Hb,value_Total Protein,value_Urea,value_WBC
0,41308,2021-05-31,200.0,25.0,26.0,0.04,26.0,11.0,NaN,NaN,...,NaN,121.0,NaN,NaN,29.9,NaN,136.0,80.0,9.7,15.98
1,41308,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56749,2020-11-24,58.0,13.0,35.0,0.02,28.0,5.0,2.22,102.0,...,NaN,150.0,NaN,NaN,37.4,NaN,NaN,63.0,12.2,6.44
3,56749,2020-11-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,178.0,NaN,NaN,36.8,NaN,NaN,NaN,NaN,NaN
4,56749,2021-07-29,125.0,354.0,30.0,0.02,29.0,10.0,2.39,NaN,...,NaN,191.0,NaN,NaN,36.3,NaN,NaN,69.0,25.5,5.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18741,2526936,2021-08-21,NaN,NaN,NaN,0.03,29.0,NaN,NaN,99.0,...,NaN,161.0,NaN,NaN,36.4,NaN,NaN,NaN,2.5,6.84
18742,2526936,2021-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,134.0,NaN,NaN,36.2,NaN,NaN,NaN,NaN,NaN
18743,2526936,2021-08-23,NaN,NaN,NaN,0.04,29.0,NaN,NaN,99.0,...,NaN,129.0,NaN,NaN,36.7,NaN,NaN,NaN,2.8,5.85
18744,2526936,2021-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,122.0,NaN,NaN,36.3,NaN,NaN,NaN,NaN,NaN


In [10]:
import re

columnNames = ['patient_id', 'effectiveDateTime']
columnNames.extend([col.replace('value_', 'vitals_') + '_avg' for col in datamatrixDf.columns.to_list() if col.startswith('value_')])
datamatrixDf = datamatrixDf.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
datamatrixDf.columns = columnNames
datamatrixDf = datamatrixDf.rename(columns={"effectiveDateTime": "measurement_date"})
datamatrixDf

,patient_id,measurement_date,vitals_ALP_avg,vitals_ALT_avg,vitals_Albumin_avg,vitals_Basophils_avg,vitals_Bicarbonate_avg,vitals_Bilirubin_avg,vitals_Calcium_avg,vitals_Chloride_avg,...,vitals_Systolic Blood Pressure Supine_avg,"vitals_Systolic Blood Pressure, Cuff_avg",vitals_Temperature Axillary_avg,vitals_Temperature Nasoesophageal_avg,vitals_Temperature Temporal_avg,vitals_Temperature Tympanic_avg,vitals_Total Hb_avg,vitals_Total Protein_avg,vitals_Urea_avg,vitals_WBC_avg
0,41308,2021-05-31,200.0,25.0,26.0,0.04,26.0,11.0,NaN,NaN,...,NaN,121.0,NaN,NaN,29.9,NaN,136.0,80.0,9.7,15.98
1,41308,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56749,2020-11-24,58.0,13.0,35.0,0.02,28.0,5.0,2.22,102.0,...,NaN,150.0,NaN,NaN,37.4,NaN,NaN,63.0,12.2,6.44
3,56749,2020-11-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,178.0,NaN,NaN,36.8,NaN,NaN,NaN,NaN,NaN
4,56749,2021-07-29,125.0,354.0,30.0,0.02,29.0,10.0,2.39,NaN,...,NaN,191.0,NaN,NaN,36.3,NaN,NaN,69.0,25.5,5.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18741,2526936,2021-08-21,NaN,NaN,NaN,0.03,29.0,NaN,NaN,99.0,...,NaN,161.0,NaN,NaN,36.4,NaN,NaN,NaN,2.5,6.84
18742,2526936,2021-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,134.0,NaN,NaN,36.2,NaN,NaN,NaN,NaN,NaN
18743,2526936,2021-08-23,NaN,NaN,NaN,0.04,29.0,NaN,NaN,99.0,...,NaN,129.0,NaN,NaN,36.7,NaN,NaN,NaN,2.8,5.85
18744,2526936,2021-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,122.0,NaN,NaN,36.3,NaN,NaN,NaN,NaN,NaN


## Save Data Matrix

In [11]:
casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'

datamatrixDf.to_csv(casestudy1DirPath + '/data_matrix.csv', index=False)

## Read Data Matrix

In [12]:
import pandas as pd

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'

datamatrixDf = pd.read_csv(casestudy1DirPath + '/data_matrix.csv')
datamatrixDf

,patient_id,measurement_date,vitals_ALP_avg,vitals_ALT_avg,vitals_Albumin_avg,vitals_Basophils_avg,vitals_Bicarbonate_avg,vitals_Bilirubin_avg,vitals_Calcium_avg,vitals_Chloride_avg,...,vitals_Systolic Blood Pressure Supine_avg,"vitals_Systolic Blood Pressure, Cuff_avg",vitals_Temperature Axillary_avg,vitals_Temperature Nasoesophageal_avg,vitals_Temperature Temporal_avg,vitals_Temperature Tympanic_avg,vitals_Total Hb_avg,vitals_Total Protein_avg,vitals_Urea_avg,vitals_WBC_avg
0,41308,2021-05-31,200.0,25.0,26.0,0.04,26.0,11.0,NaN,NaN,...,NaN,121.0,NaN,NaN,29.9,NaN,136.0,80.0,9.7,15.98
1,41308,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56749,2020-11-24,58.0,13.0,35.0,0.02,28.0,5.0,2.22,102.0,...,NaN,150.0,NaN,NaN,37.4,NaN,NaN,63.0,12.2,6.44
3,56749,2020-11-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,178.0,NaN,NaN,36.8,NaN,NaN,NaN,NaN,NaN
4,56749,2021-07-29,125.0,354.0,30.0,0.02,29.0,10.0,2.39,NaN,...,NaN,191.0,NaN,NaN,36.3,NaN,NaN,69.0,25.5,5.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18741,2526936,2021-08-21,NaN,NaN,NaN,0.03,29.0,NaN,NaN,99.0,...,NaN,161.0,NaN,NaN,36.4,NaN,NaN,NaN,2.5,6.84
18742,2526936,2021-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,134.0,NaN,NaN,36.2,NaN,NaN,NaN,NaN,NaN
18743,2526936,2021-08-23,NaN,NaN,NaN,0.04,29.0,NaN,NaN,99.0,...,NaN,129.0,NaN,NaN,36.7,NaN,NaN,NaN,2.8,5.85
18744,2526936,2021-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,122.0,NaN,NaN,36.3,NaN,NaN,NaN,NaN,NaN


## Coverage analysis

In [13]:
import os

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'


os.system(
    '''cd $EHR_QC_PREPROCESS_BASE;'''
    +
    '''.venv/bin/python -m ehrqc.coverage.Coverage ''' + casestudy1DirPath + '/data_matrix.csv' + ''' 100 -i "patient_id" -d -p 30 -sp ''' + casestudy1DirPath + '/data_matrix_dense.csv'
    )


2024-08-28 04:14:15,585 - EHR-QC - INFO - Parsing command line arguments
2024-08-28 04:14:15,586 - EHR-QC - INFO - Start!!
2024-08-28 04:14:15,586 - EHR-QC - INFO - args.source_file: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/data_matrix.csv
2024-08-28 04:14:15,586 - EHR-QC - INFO - args.chunksize: 100
2024-08-28 04:14:15,586 - EHR-QC - INFO - args.id_columns: ['patient_id']
2024-08-28 04:14:15,586 - EHR-QC - INFO - args.drop: True
2024-08-28 04:14:15,586 - EHR-QC - INFO - args.percentage: 30.0
2024-08-28 04:14:15,586 - EHR-QC - INFO - args.save_path: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/data_matrix_dense.csv
2024-08-28 04:14:15,586 - EHR-QC - INFO - Calculating Missingnes
2024-08-28 04:14:16,283 - EHR-QC - INFO - Missingness Report
2024-08-28 04:14:16,286 - EHR-QC - INFO - 

                                       column_name  missing_count  \
0                                 measurement_date              0   
1                                   v

0

## Impute Missing Data

In [14]:
import os

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'


os.system(
    '''cd $EHR_QC_PREPROCESS_BASE;'''
    +
    '''.venv/bin/python -m ehrqc.impute.Impute ''' + casestudy1DirPath + '''/data_matrix_dense.csv -ac 'impute' -sp ''' + casestudy1DirPath + '''/data_matrix_imputed.csv -al mean -c "vitals_ALP_avg" "vitals_ALT_avg" "vitals_Albumin_avg" "vitals_Basophils_avg" "vitals_Bicarbonate_avg" "vitals_Bilirubin_avg" "vitals_Calcium_avg" "vitals_Chloride_avg" "vitals_Creatinine_avg" "vitals_Eosinophils_avg" "vitals_GGT_avg" "vitals_Globulin_avg" "vitals_Hb_avg" "vitals_Hct_avg" "vitals_Heart Rate Monitored_avg" "vitals_Lymphocytes_avg" "vitals_MCH_avg" "vitals_MCHC_avg" "vitals_MCV_avg" "vitals_Magnesium Level_avg" "vitals_Monocytes_avg" "vitals_Neutrophils_avg" "vitals_Phosphate Level_avg" "vitals_Platelets_avg" "vitals_Potassium_avg" "vitals_RBC_avg" "vitals_RDW_avg" "vitals_Respiratory Rate_avg" "vitals_Sodium_avg" "vitals_SpO2_avg" "vitals_Temperature Temporal_avg" "vitals_Total Protein_avg" "vitals_Urea_avg" "vitals_WBC_avg" "vitals_Systolic Blood Pressure, Cuff_avg" "vitals_Diastolic Blood Pressure, Cuff_avg" "vitals_Mean Arterial Pressure, Cuff_avg"'''
    )


2024-08-28 04:14:17,392 - EHR-QC - INFO - Parsing command line arguments
2024-08-28 04:14:17,393 - EHR-QC - INFO - args.action: impute
2024-08-28 04:14:17,393 - EHR-QC - INFO - args.source_path: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/data_matrix_dense.csv
2024-08-28 04:14:17,393 - EHR-QC - INFO - args.percentage: None
2024-08-28 04:14:17,393 - EHR-QC - INFO - args.save_path: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/data_matrix_imputed.csv
2024-08-28 04:14:17,393 - EHR-QC - INFO - args.algorithm: mean
2024-08-28 04:14:17,393 - EHR-QC - INFO - args.columns: ['vitals_ALP_avg', 'vitals_ALT_avg', 'vitals_Albumin_avg', 'vitals_Basophils_avg', 'vitals_Bicarbonate_avg', 'vitals_Bilirubin_avg', 'vitals_Calcium_avg', 'vitals_Chloride_avg', 'vitals_Creatinine_avg', 'vitals_Eosinophils_avg', 'vitals_GGT_avg', 'vitals_Globulin_avg', 'vitals_Hb_avg', 'vitals_Hct_avg', 'vitals_Heart Rate Monitored_avg', 'vitals_Lymphocytes_avg', 'vitals_MCH_avg', 'vitals_MCHC_avg

0

## Remove outliers

In [15]:
import os

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'


os.system(
    '''cd $EHR_QC_PREPROCESS_BASE;'''
    +
    '''.venv/bin/python -m ehrqc.plot.OutlierIsolationForest ''' + casestudy1DirPath + '''/data_matrix_imputed.csv ''' + casestudy1DirPath + '''/data_matrix_clean.csv clean -c "vitals_ALP_avg" "vitals_ALT_avg" "vitals_Albumin_avg" "vitals_Basophils_avg" "vitals_Bicarbonate_avg" "vitals_Bilirubin_avg" "vitals_Calcium_avg" "vitals_Chloride_avg" "vitals_Creatinine_avg" "vitals_Eosinophils_avg" "vitals_GGT_avg" "vitals_Globulin_avg" "vitals_Hb_avg" "vitals_Hct_avg" "vitals_Heart Rate Monitored_avg" "vitals_Lymphocytes_avg" "vitals_MCH_avg" "vitals_MCHC_avg" "vitals_MCV_avg" "vitals_Magnesium Level_avg" "vitals_Monocytes_avg" "vitals_Neutrophils_avg" "vitals_Phosphate Level_avg" "vitals_Platelets_avg" "vitals_Potassium_avg" "vitals_RBC_avg" "vitals_RDW_avg" "vitals_Respiratory Rate_avg" "vitals_Sodium_avg" "vitals_SpO2_avg" "vitals_Temperature Temporal_avg" "vitals_Total Protein_avg" "vitals_Urea_avg" "vitals_WBC_avg" "vitals_Systolic Blood Pressure, Cuff_avg" "vitals_Diastolic Blood Pressure, Cuff_avg" "vitals_Mean Arterial Pressure, Cuff_avg"'''
    )


2024-08-28 04:14:18,362 - EHR-QC - INFO - Parsing command line arguments
2024-08-28 04:14:18,363 - EHR-QC - INFO - args.source_file: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/data_matrix_imputed.csv
2024-08-28 04:14:18,363 - EHR-QC - INFO - args.save_file: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/data_matrix_clean.csv
2024-08-28 04:14:18,363 - EHR-QC - INFO - args.action: clean
2024-08-28 04:14:18,363 - EHR-QC - INFO - args.columns: ['vitals_ALP_avg', 'vitals_ALT_avg', 'vitals_Albumin_avg', 'vitals_Basophils_avg', 'vitals_Bicarbonate_avg', 'vitals_Bilirubin_avg', 'vitals_Calcium_avg', 'vitals_Chloride_avg', 'vitals_Creatinine_avg', 'vitals_Eosinophils_avg', 'vitals_GGT_avg', 'vitals_Globulin_avg', 'vitals_Hb_avg', 'vitals_Hct_avg', 'vitals_Heart Rate Monitored_avg', 'vitals_Lymphocytes_avg', 'vitals_MCH_avg', 'vitals_MCHC_avg', 'vitals_MCV_avg', 'vitals_Magnesium Level_avg', 'vitals_Monocytes_avg', 'vitals_Neutrophils_avg', 'vitals_Phosphate Level_avg

/home/vmadmin/workspace/EHR-QC-Preprocess/ehrqc/plot/OutlierIsolationForest.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correctedDf.drop(columns=['outlier_scores'], inplace=True)


2024-08-28 04:14:18,606 - EHR-QC - INFO - Saving the corrected file
2024-08-28 04:14:18,810 - EHR-QC - INFO - Done!!


0

## Add target labels

### Read cleaned data

In [16]:
import pandas as pd

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'


dataDf = pd.read_csv(casestudy1DirPath + '''/data_matrix_clean.csv''')
dataDf

,patient_id,measurement_date,vitals_ALP_avg,vitals_ALT_avg,vitals_Albumin_avg,vitals_Basophils_avg,vitals_Bicarbonate_avg,vitals_Bilirubin_avg,vitals_Calcium_avg,vitals_Chloride_avg,...,vitals_Respiratory Rate_avg,vitals_Sodium_avg,vitals_SpO2_avg,vitals_Temperature Temporal_avg,vitals_Total Protein_avg,vitals_Urea_avg,vitals_WBC_avg,"vitals_Systolic Blood Pressure, Cuff_avg","vitals_Diastolic Blood Pressure, Cuff_avg","vitals_Mean Arterial Pressure, Cuff_avg"
0,41308,2021-05-31,200.000000,25.000000,26.000000,0.04,26.0,11.000000,2.191763,103.388509,...,26.0,137.410734,95.0,29.9,80.000000,9.7,15.98,121.0,55.0,82.000000
1,56749,2020-11-24,58.000000,13.000000,35.000000,0.02,28.0,5.000000,2.220000,102.000000,...,18.0,136.000000,100.0,37.4,63.000000,12.2,6.44,150.0,87.0,104.000000
2,56749,2021-07-29,125.000000,354.000000,30.000000,0.02,29.0,10.000000,2.390000,103.388509,...,15.0,144.000000,98.0,36.3,69.000000,25.5,5.18,191.0,85.0,122.000000
3,56749,2021-07-30,107.000000,264.000000,26.000000,0.03,26.0,8.000000,2.200000,103.000000,...,18.0,141.000000,95.0,36.2,59.000000,23.0,4.72,132.0,57.0,113.000000
4,56749,2021-07-31,127.025688,35.740125,27.285302,0.01,27.0,16.974972,2.220000,106.000000,...,18.0,142.000000,97.0,35.9,65.951456,15.8,6.67,123.0,56.0,114.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11682,2526936,2021-08-17,127.025688,35.740125,27.285302,0.01,33.0,16.974972,2.180000,98.000000,...,16.0,136.000000,96.0,36.5,65.951456,2.8,9.04,147.0,89.0,84.952515
11683,2526936,2021-08-18,188.000000,8.000000,19.000000,0.04,30.0,13.000000,2.150000,101.000000,...,20.0,137.000000,97.0,36.8,61.000000,2.6,9.34,165.0,85.0,106.000000
11684,2526936,2021-08-19,176.000000,8.000000,20.000000,0.02,27.0,15.000000,2.290000,100.000000,...,12.0,135.000000,98.0,36.5,66.000000,2.4,9.48,177.0,91.0,107.000000
11685,2526936,2021-08-21,127.025688,35.740125,27.285302,0.03,29.0,16.974972,2.191763,99.000000,...,16.0,134.000000,97.0,36.4,65.951456,2.5,6.84,161.0,94.0,113.000000


### Read labels

In [17]:
import pandas as pd

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'


labelsDf = pd.read_csv(casestudy1DirPath + '''/los_labels.csv''')
labelsDf = labelsDf.groupby('person_id').max().reset_index()
labelsDf

,person_id,los_gt_7_days,los_gt_14_days,los_gt_30_days,los_gt_60_days,los_gt_90_days
0,18849,0,0,0,0,0
1,41308,0,0,0,0,0
2,50056,1,1,0,0,0
3,51211,0,0,0,0,0
4,51541,1,0,0,0,0
...,...,...,...,...,...,...
2530,2678697,1,1,0,0,0
2531,2679272,1,1,0,0,0
2532,2680486,1,0,0,0,0
2533,2682433,0,0,0,0,0


## Add admission time

### Read admission file

In [35]:
import pandas as pd

casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'


admissionsDf = pd.read_csv(casestudy1DirPath + '''/admissions.csv''')
admissionsDf.admittime = pd.to_datetime(admissionsDf.admittime, format='%Y-%m-%d %H:%M:%S.%f').dt.date
# admissionsDf = admissionsDf[['person_id', 'admittime']].drop_duplicates()
admissionsDf = admissionsDf[['person_id', 'admittime']].groupby('person_id').max().reset_index()
admissionsDf

,person_id,admittime
0,18849,2019-09-01
1,41308,2021-05-31
2,49412,2022-02-28
3,50056,2020-02-08
4,51211,2023-04-01
...,...,...
5213,2680034,2023-03-14
5214,2680486,2023-03-16
5215,2681377,2023-03-21
5216,2682433,2023-03-24


### Merge both dataframes

In [36]:
labelsMergedDf = dataDf.merge(
            labelsDf,
            how='inner',
            left_on='patient_id',
            right_on='person_id'
        ).drop(
            columns=['patient_id']
        ).merge(
            admissionsDf,
            how='inner',
            on='person_id'
        )
labelsMergedDf

,measurement_date,vitals_ALP_avg,vitals_ALT_avg,vitals_Albumin_avg,vitals_Basophils_avg,vitals_Bicarbonate_avg,vitals_Bilirubin_avg,vitals_Calcium_avg,vitals_Chloride_avg,vitals_Creatinine_avg,...,"vitals_Systolic Blood Pressure, Cuff_avg","vitals_Diastolic Blood Pressure, Cuff_avg","vitals_Mean Arterial Pressure, Cuff_avg",person_id,los_gt_7_days,los_gt_14_days,los_gt_30_days,los_gt_60_days,los_gt_90_days,admittime
0,2021-05-31,200.000000,25.000000,26.000000,0.04,26.0,11.000000,2.191763,103.388509,85.0,...,121.0,55.0,82.000000,41308,0,0,0,0,0,2021-05-31
1,2020-11-24,58.000000,13.000000,35.000000,0.02,28.0,5.000000,2.220000,102.000000,91.0,...,150.0,87.0,104.000000,56749,1,1,1,0,0,2021-07-30
2,2021-07-29,125.000000,354.000000,30.000000,0.02,29.0,10.000000,2.390000,103.388509,103.0,...,191.0,85.0,122.000000,56749,1,1,1,0,0,2021-07-30
3,2021-07-30,107.000000,264.000000,26.000000,0.03,26.0,8.000000,2.200000,103.000000,101.0,...,132.0,57.0,113.000000,56749,1,1,1,0,0,2021-07-30
4,2021-07-31,127.025688,35.740125,27.285302,0.01,27.0,16.974972,2.220000,106.000000,103.0,...,123.0,56.0,114.000000,56749,1,1,1,0,0,2021-07-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9725,2021-08-17,127.025688,35.740125,27.285302,0.01,33.0,16.974972,2.180000,98.000000,68.0,...,147.0,89.0,84.952515,2526936,1,1,1,0,0,2021-08-15
9726,2021-08-18,188.000000,8.000000,19.000000,0.04,30.0,13.000000,2.150000,101.000000,58.0,...,165.0,85.0,106.000000,2526936,1,1,1,0,0,2021-08-15
9727,2021-08-19,176.000000,8.000000,20.000000,0.02,27.0,15.000000,2.290000,100.000000,53.0,...,177.0,91.0,107.000000,2526936,1,1,1,0,0,2021-08-15
9728,2021-08-21,127.025688,35.740125,27.285302,0.03,29.0,16.974972,2.191763,99.000000,61.0,...,161.0,94.0,113.000000,2526936,1,1,1,0,0,2021-08-15


## Save final datamatrix

In [37]:
casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'

labelsMergedDf.to_csv(casestudy1DirPath + '/data_matrix_final.csv', index=False)

## Generate final report (summary report)

In [23]:
import os


casestudy1DirPath = '/home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1'

os.system(
    'cd ' + os.environ['EHR_QC_PREPROCESS_BASE'] + ';'
    +
    '''.venv/bin/python -m ehrqc.plot.ExplorationGraphsPdf'''
    +
    ' ' + casestudy1DirPath + '' + '/summary_report.pdf'
    +
    ''' -c '{'''
    +
    ''' "ALP" : ["vitals_ALP_avg", "vitals_ALP_avg", "vitals_ALP_avg"],'''
    +
    ''' "ALT" : ["vitals_ALT_avg", "vitals_ALT_avg", "vitals_ALT_avg"],'''
    +
    ''' "Albumin" : ["vitals_Albumin_avg", "vitals_Albumin_avg", "vitals_Albumin_avg"],'''
    +
    ''' "Basophils" : ["vitals_Basophils_avg", "vitals_Basophils_avg", "vitals_Basophils_avg"],'''
    +
    ''' "Bicarbonate" : ["vitals_Bicarbonate_avg", "vitals_Bicarbonate_avg", "vitals_Bicarbonate_avg"],'''
    +
    ''' "Bilirubin" : ["vitals_Bilirubin_avg", "vitals_Bilirubin_avg", "vitals_Bilirubin_avg"],'''
    +
    ''' "Calcium" : ["vitals_Calcium_avg", "vitals_Calcium_avg", "vitals_Calcium_avg"],'''
    +
    ''' "Chloride" : ["vitals_Chloride_avg", "vitals_Chloride_avg", "vitals_Chloride_avg"],'''
    +
    ''' "Creatinine" : ["vitals_Creatinine_avg", "vitals_Creatinine_avg", "vitals_Creatinine_avg"],'''
    +
    ''' "Eosinophils" : ["vitals_Eosinophils_avg", "vitals_Eosinophils_avg", "vitals_Eosinophils_avg"],'''
    +
    ''' "GGT" : ["vitals_GGT_avg", "vitals_GGT_avg", "vitals_GGT_avg"],'''
    +
    ''' "Globulin" : ["vitals_Globulin_avg", "vitals_Globulin_avg", "vitals_Globulin_avg"],'''
    +
    ''' "Hb" : ["vitals_Hb_avg", "vitals_Hb_avg", "vitals_Hb_avg"],'''
    +
    ''' "Hct" : ["vitals_Hct_avg", "vitals_Hct_avg", "vitals_Hct_avg"],'''
    +
    ''' "Lymphocytes" : ["vitals_Lymphocytes_avg", "vitals_Lymphocytes_avg", "vitals_Lymphocytes_avg"],'''
    +
    ''' "MCH" : ["vitals_MCH_avg", "vitals_MCH_avg", "vitals_MCH_avg"],'''
    +
    ''' "MCHC" : ["vitals_MCHC_avg", "vitals_MCHC_avg", "vitals_MCHC_avg"],'''
    +
    ''' "MCV" : ["vitals_MCV_avg", "vitals_MCV_avg", "vitals_MCV_avg"],'''
    +
    ''' "Monocytes" : ["vitals_Monocytes_avg", "vitals_Monocytes_avg", "vitals_Monocytes_avg"],'''
    +
    ''' "Neutrophils" : ["vitals_Neutrophils_avg", "vitals_Neutrophils_avg", "vitals_Neutrophils_avg"],'''
    +
    ''' "Platelets" : ["vitals_Platelets_avg", "vitals_Platelets_avg", "vitals_Platelets_avg"],'''
    +
    ''' "Potassium" : ["vitals_Potassium_avg", "vitals_Potassium_avg", "vitals_Potassium_avg"],'''
    +
    ''' "RBC" : ["vitals_RBC_avg", "vitals_RBC_avg", "vitals_RBC_avg"],'''
    +
    ''' "RDW" : ["vitals_RDW_avg", "vitals_RDW_avg", "vitals_RDW_avg"],'''
    +
    ''' "Sodium" : ["vitals_Sodium_avg", "vitals_Sodium_avg", "vitals_Sodium_avg"],'''
    +
    ''' "SpO2" : ["vitals_SpO2_avg", "vitals_SpO2_avg", "vitals_SpO2_avg"],'''
    +
    ''' "Urea" : ["vitals_Urea_avg", "vitals_Urea_avg", "vitals_Urea_avg"],'''
    +
    ''' "WBC" : ["vitals_WBC_avg", "vitals_WBC_avg", "vitals_WBC_avg"]'''
    +
    ''' }' '''
    +
    ''' -sf ''' + casestudy1DirPath + '''/data_matrix_dense.csv ''' + casestudy1DirPath + '''/data_matrix_imputed.csv ''' + casestudy1DirPath + '''/data_matrix_final.csv'''
    +
    ''' -l "Raw" "Imputed" "Cleaned"'''
    )


2024-09-02 07:25:54,064 - EHR-QC - INFO - Parsing command line arguments
2024-09-02 07:25:54,065 - EHR-QC - INFO - args.save_path: /home/vmadmin/workspace/EHR-Int-Analysis/data/case_study_1/summary_report.pdf
2024-09-02 07:25:54,065 - EHR-QC - INFO - args.columns: {'ALP': ['vitals_ALP_avg', 'vitals_ALP_avg', 'vitals_ALP_avg'], 'ALT': ['vitals_ALT_avg', 'vitals_ALT_avg', 'vitals_ALT_avg'], 'Albumin': ['vitals_Albumin_avg', 'vitals_Albumin_avg', 'vitals_Albumin_avg'], 'Basophils': ['vitals_Basophils_avg', 'vitals_Basophils_avg', 'vitals_Basophils_avg'], 'Bicarbonate': ['vitals_Bicarbonate_avg', 'vitals_Bicarbonate_avg', 'vitals_Bicarbonate_avg'], 'Bilirubin': ['vitals_Bilirubin_avg', 'vitals_Bilirubin_avg', 'vitals_Bilirubin_avg'], 'Calcium': ['vitals_Calcium_avg', 'vitals_Calcium_avg', 'vitals_Calcium_avg'], 'Chloride': ['vitals_Chloride_avg', 'vitals_Chloride_avg', 'vitals_Chloride_avg'], 'Creatinine': ['vitals_Creatinine_avg', 'vitals_Creatinine_avg', 'vitals_Creatinine_avg'], 'Eosino

/home/vmadmin/workspace/EHR-QC-Preprocess/ehrqc/plot/ExplorationGraphsPdf.py:38: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


2024-09-02 07:26:00,910 - EHR-QC - INFO - Plotting: Potassium
2024-09-02 07:26:01,288 - EHR-QC - INFO - Plotting: RBC
2024-09-02 07:26:01,661 - EHR-QC - INFO - Plotting: RDW
2024-09-02 07:26:01,967 - EHR-QC - INFO - Plotting: Sodium
2024-09-02 07:26:02,327 - EHR-QC - INFO - Plotting: SpO2
2024-09-02 07:26:02,689 - EHR-QC - INFO - Plotting: Urea
2024-09-02 07:26:03,010 - EHR-QC - INFO - Plotting: WBC
2024-09-02 07:26:03,373 - EHR-QC - INFO - Saving the pdf
2024-09-02 07:26:03,379 - EHR-QC - INFO - Done!!


0